In [9]:
import json
import urllib.request
import urllib3
import time
from datetime import timedelta
import boto3
import logging

json_data = open('config.json')
j_obj = json.load(json_data)

state = j_obj['state']
team = j_obj['team']
link = j_obj['link']
AWSAccess = j_obj['AWSAccess']
AWSSecret = j_obj['AWSSecret']
email = j_obj['notificationEmail']


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logfile = time.strftime("%Y-%m-%d_%H:%M:%S" + ".log")

# create a file handler
handler = logging.FileHandler(logfile)
handler.setLevel(logging.INFO)
# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(handler)

logger.info('Hello!!!!')

filename = state + '_' + time.strftime("%d%m%Y") + '_WBAN_53138.csv'
# filename = state + '_' + time.strftime("%d%m%Y") + '_WBAN_53138.csv'
print(filename)

file = urllib.request.urlretrieve(link, filename)


with open('link.json', "r") as jsonFile:
    data = json.load(jsonFile)

data[filename] = link

with open('link.json', 'w') as json_file:
    json.dump(data, json_file)


INFO:__main__:Hello!!!!


NV_23062017_WBAN_53138.csv


HTTPError: HTTP Error 404: Not Found

In [ ]:
# create connection to Amazon s3
logger.info('Creating connection to Amazon s3!')
client = boto3.client('s3',
                      aws_access_key_id=AWSAccess, 
                      aws_secret_access_key=AWSSecret)
s3 = boto3.resource('s3',
                   aws_access_key_id=AWSAccess, 
                      aws_secret_access_key=AWSSecret)

In [ ]:
import botocore
BucketName = 'Team10_NV_Assignment1'
bucket = s3.Bucket(BucketName)
exists = True
try:
    s3.meta.client.head_bucket(Bucket=BucketName)
except botocore.exceptions.ClientError as e:
    error_code = int(e.response['Error']['Code'])
    if error_code == 404:
        logger.info(BucketName + " does not exist!")
        exists = False
    else:
        logger.info(BucketName + " exists!")
if exists == False:
    logger.info("Creating " + BucketName)
    client.create_bucket(Bucket=BucketName)

In [ ]:
import fnmatch
import pandas as pd
import urllib.request
import os

s3 = boto3.resource('s3',
                   aws_access_key_id=AWSAccess, 
                      aws_secret_access_key=AWSSecret)
flag = False
oldfilecount = 0
oldfilename = ''
bucket = s3.Bucket(BucketName)
for obj in bucket.objects.all():
    if fnmatch.fnmatch(obj.key, '*_WBAN_53138.csv'):
        oldfilecount += 1
        if obj.key > oldfilename:
            oldfilename = obj.key       
        
    if obj.key == filename:
        logger.info(filename + ' already exists!')
        flag = True
        
# file does not exist(1. no raw file 2. one old raw file)
if flag == False and oldfilecount == 0:
    logger.info(filename + ' does not exists!')
    logger.info('Uploading file!')
    s3.meta.client.upload_file(filename, BucketName, filename)
#2. one old raw file merge new file with old file
elif flag == False and oldfilecount > 0:
    oldfile = pd.read_csv(oldfilename)
    newfile = pd.read_csv(filename)
    file = pd.concat([oldfile, newfile])
    file.to_csv(filename, sep='\t')
    s3.meta.client.upload_file(filename, BucketName, filename)
    
elif flag == True:
    print(filename + ' is existing!')
    
# download file to jupyter file system
logger.info('Downloading ' + filename)
os.remove(filename)
s3.meta.client.download_file(BucketName, filename, filename)

In [ ]:
# filename = 'config.json'

# with open(filename, "r") as jsonFile:
#     data = json.load(jsonFile)
# data['link'] = "https://www.ncei.noaa.gov/orders/cdo/996880.csv"
# with open(filename, "w") as jsonFile:
#     json.dump(data, jsonFile)